In [1]:
import os
import sys
import inspect
import tqdm
import warnings
import argparse
import datetime

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir) + "/src/"
sys.path.insert(0, parentdir) 
from modeling import *
from utils import NpEncoder
%load_ext autoreload
%autoreload 2


In [10]:
type_ = "R"

# load data and get player
datafile = "../data/raw/data_%s.json"%(type_)
print("data file\t", datafile)
with open(datafile, 'r') as f:
    data = json.load(f)
player = list(data.keys())[0]


# multi-start parameters
params = {
"inv_temp": 0.1, 
}

data file	 ../data/raw/data_R.json


In [11]:
filter_fn = filter_depth
value_fn = value_max
games = format_games(data[player]["data"])

In [12]:
model = Model(filter_fn, value_fn, variant =type_)

In [13]:
game_data = preprocess_data(games)
pov_array = make_pov_array(game_data.boards, game_data.paths)
filter_pov_array = model.filter_fn(pov_array)
ml, ml_params = model.max_likelihood_estimation(params, filter_pov_array, game_data.choose_left)

In [51]:
choose_left = game_data.choose_left
p_left = model.get_p_left(params, filter_pov_array)
log_likelihood = choose_left * np.log(p_left) + (1 - choose_left) * np.log(1 - p_left)
log_likelihood_player = log_likelihood.sum(["games", "trials", "conditions"], keep_attrs = ["conditions"])
ml = log_likelihood_player.max("filter_params")
# mle = argmax_random_tiebreaker(log_likelihood_player)
# copy_and_update(params, {"filter_params": mle})

<xarray.DataArray ()> Size: 8B
array(-727.80453959)

In [45]:
ml.sum("conditions")

ValueError: 'conditions' not found in array dimensions ()